<a href="https://colab.research.google.com/github/sagihaider/EEG_Deep/blob/master/main_Single_Trial_Classification_PatientData_Binary_NSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sagihaider/EEG_Deep.git

Cloning into 'EEG_Deep'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 207 (delta 28), reused 13 (delta 5), pack-reused 153
Receiving objects: 100% (207/207), 858.77 MiB | 38.70 MiB/s, done.
Resolving deltas: 100% (81/81), done.
Checking out files: 100% (60/60), done.


In [55]:
import scipy.io as spio
import numpy as np
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from EEG_Deep.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_Patient
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

!pip install neural_structured_learning
import tensorflow as tf
import neural_structured_learning as nsl

ValueError: ignored

In [0]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [0]:
class HParams(object):
  """Hyperparameters used for training."""
  def __init__(self):
    ### eval parameters
    self.eval_steps = 1  # All instances in the test set are evaluated.

HPARAMS = HParams()

In [76]:
from numpy import zeros
K.clear_session()
cols = 1
rows = 10
acc_all = zeros([rows, cols])
loss_all = zeros([rows, cols])
X_tr_c12 = np.empty([80, 12, 4096])
X_ts_c12 = np.empty([80, 12, 4096])
result=[]

from itertools import combinations 
comb = combinations([1, 2], 2) 
  # Print the obtained combinations 
bincomb=[]
for i in list(comb): 
    bincomb.append(i)

for x in range(1,2):
  fName = 'EEG_Deep/Data2A/parsed_P0' + str(x) + 'T.mat'  # Load Data
  print(fName)
  mat = spio.loadmat(fName)
  r_X_tr = mat['RawEEGData']
  r_y_tr = mat['Labels']

  ### Filter Data ###
  for t in range(r_X_tr.shape[0]):
    tril = r_X_tr[t,:,:]
    # tril = tril.transpose()
    tril_filtered = butter_bandpass_filter(tril, 
                                              lowcut=8, 
                                              highcut=24, 
                                              fs=512,
                                              order=4)
    # tril_filtered = tril_filtered.transpose()
    X_tr_c12[t,:,:] = tril_filtered

  print("Filtering of Training Data Finished")
  ## Test Data Load 

  fName = 'EEG_Deep/Data2A/parsed_P0' + str(x) + 'E.mat'  # Load Data
  print(fName)
  mat = spio.loadmat(fName)
  r_X_ts = mat['RawEEGData']
  r_y_ts = mat['Labels']
  for t in range(r_X_ts.shape[0]):
    tril = r_X_ts[t,:,:]
    # tril = tril.transpose()
    tril_filtered = butter_bandpass_filter(tril, 
                                              lowcut=8, 
                                              highcut=24, 
                                              fs=512,
                                              order=4)
    # tril_filtered = tril_filtered.transpose()
    X_ts_c12[t,:,:] = tril_filtered
  
  print("Filtering of Testing Data Finished")    

  for k, com in enumerate(bincomb):
      print(com)
      # # Find labels to specific class
      print("Finding labels in training data")
      class1indx = list(np.where(r_y_tr == com[0]))
      class2indx = list(np.where(r_y_tr == com[1]))
      c1=list(class1indx[0])
      c2=list(class2indx[0])
      y_tr_c12 = c1 + c2
      y_tr_c12.sort()
      # print(y_tr_c12)
      x_tr_12 = X_tr_c12[y_tr_c12,:,:]
      y_tr_12 = r_y_tr[y_tr_c12]
      # print(np.shape(x_tr_12))
      # print(np.shape(y_tr_12))
      # # Find labels to specific class
      print("Finding labels in testing data")
      class1indx = list(np.where(r_y_ts == com[0]))
      class2indx = list(np.where(r_y_ts == com[1]))
      c1=list(class1indx[0])
      c2=list(class2indx[0])
      y_ts_c12 = c1 + c2
      y_ts_c12.sort()
      # print(y_ts_c12)
      x_ts_12 = X_ts_c12[y_ts_c12,:,:]
      y_ts_12 = r_y_ts[y_ts_c12]
      # print(np.shape(x_ts_12))
      # print(np.shape(y_ts_12))
      del class1indx, class2indx, c1, c2

      # shuffle the training data
      indices = np.arange(x_tr_12.shape[0])
      np.random.shuffle(indices)
      x_tr_12 = x_tr_12[indices]
      y_tr_12 = y_tr_12[indices]

      # split data of each subject in training and validation
      X_train = x_tr_12[0:60,:,2560:4096]
      Y_train = y_tr_12[0:60].ravel()
      X_val   = x_tr_12[60:,:,2560:4096]
      Y_val   = y_tr_12[60:].ravel()
      print(Y_val)
      print(np.shape(X_train))
      print(np.shape(Y_train))
      print(np.shape(X_val))
      print(np.shape(Y_val))
  
      # convert labels to one-hot encodings.
      Y_train      = np_utils.to_categorical(Y_train-1, num_classes=4)
      Y_val       = np_utils.to_categorical(Y_val-1, num_classes=4)
      print(Y_val)

      kernels, chans, samples = 1, 12, 1536
      # convert data to NCHW (trials, kernels, channels, samples) format. Data 
      # contains 22 channels and 500 time-points. Set the number of kernels to 1.
      X_train = X_train.reshape(X_train.shape[0], kernels, chans, samples)
      X_val   = X_val.reshape(X_val.shape[0], kernels, chans, samples)
   
      print('X_train shape:', X_train.shape)
      print(X_train.shape[0], 'train samples')
      print(X_val.shape[0], 'val samples')

      X_test      = x_ts_12[:,:,2560:4096]
      Y_test      = y_ts_12[:]
      print(np.shape(X_test))
      print(np.shape(Y_test))

      #convert labels to one-hot encodings.
      Y_test      = np_utils.to_categorical(Y_test-1, num_classes=4)

      # convert data to NCHW (trials, kernels, channels, samples) format. Data 
      # contains 22 channels and 500 time-points. Set the number of kernels to 1.
      X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

      print('X_train shape:', X_test.shape)
      print(X_test.shape[0], 'train samples')


      ##### Learning without NSL ###########################################
      
      # # Create a base model -- sequential, functional, or subclass.
      # model = tf.keras.Sequential([
      #                              tf.keras.Input((1, 12, 1536), name='feature'),
      #                              tf.keras.layers.Flatten(),
      #                              tf.keras.layers.Dense(64, activation=tf.nn.relu),
      #                              tf.keras.layers.Dense(2, activation=tf.nn.softmax)
      #                              ])
      # model.summary()
      
      # # Compile, train, and evaluate.
      # model.compile(optimizer='adam',
      #                   loss='categorical_crossentropy',
      #                   metrics=['accuracy'])
      
      # history = model.fit(X_train, Y_train, batch_size = 32, epochs = 500, 
      #                     verbose = 2, validation_data=(X_val, Y_val)
      #                     )

      # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
      # model configurations may do better, but this is a good starting point)

      ##### Learning without NSL ################################################
    #   model = EEGNet(nb_classes = 4, Chans = 12, Samples = 1536,
    #                  dropoutRate = 0.5, kernLength = 25, F1 = 8, 
    #                  D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')

    #  # Compile, train, and evaluate.
    #   model.compile(optimizer='adam',
    #                     loss='categorical_crossentropy',
    #                     metrics=['accuracy'])
      
    #   history = model.fit(X_train, Y_train, batch_size = 32, epochs = 500, 
    #                           validation_data=(X_val, Y_val)
    #                           )


      ##### Learning with NSL ################################################
      # Wrap the model with adversarial regularization.

      model = EEGNet(nb_classes = 4, Chans = 12, Samples = 1536,
                     dropoutRate = 0.5, kernLength = 25, F1 = 8,
                     D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')
            
      adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
      adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)
      
      # Compile, train, and evaluate.
      adv_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
      batch_size = 32

      
      X_train = tf.convert_to_tensor(X_train, tf.float32)
      X_test = tf.convert_to_tensor(X_test, tf.float32)
      X_val = tf.convert_to_tensor(X_val, tf.float32)
      Y_train = tf.convert_to_tensor(Y_train)
      Y_test = tf.convert_to_tensor(Y_test)
      Y_val = tf.convert_to_tensor(Y_val)

      train_data = tf.data.Dataset.from_tensor_slices(
          {'input': X_train, 'label': Y_train}).batch(batch_size)
      val_data = tf.data.Dataset.from_tensor_slices(
          {'input': X_val, 'label': X_val}).batch(batch_size)
      val_steps = X_val.shape[0] // batch_size

      # train_data = {'input': X_train, 'label': Y_train}
      # val_data = {'input': X_val, 'label': X_val}
      # val_steps = X_val.shape[0] // batch_size
      
            
      adv_model.fit(train_data, 
                    epochs=100, verbose=1)
      
      # adv_model.fit(train_data, validation_data=val_data, steps_per_epoch = 10,
      #         validation_steps=val_steps, epochs=2, verbose=1)
      
      # history = adv_model.fit({'feature': X_train, 'label': Y_train}, batch_size = 32,
      #                     epochs = 500
      #                     # validation_data=({'feature': X_val, 'label': Y_val})
      #                     )
      

      ##### Plot with NSL ################################################

      # # Plot training & validation accuracy values
      # plt.plot(history.history['acc'])
      # plt.plot(history.history['val_acc'])
      # plt.title('Model accuracy')
      # plt.ylabel('Accuracy')
      # plt.xlabel('Epoch')
      # plt.legend(['Train', 'Test'], loc='upper left')
      # plt.show()
      # figName = 'Accuracy_A0' + str(x) + '.png'  
      # plt.savefig(figName)

      print('\n# Evaluate on test data')
      results = adv_model.evaluate({'feature': X_test, 'label': Y_test})
      # results = adv_model.evaluate(X_test, steps = HPARAMS.eval_steps)
      print('test loss, test acc:', results)

      loss_all[x - 1, k-1] = results[0]
      acc_all[x - 1, k-1] = results[1]

      from keras import backend as K 
      # Do some code, e.g. train and save model
      K.clear_session()

print(loss_all)
print(acc_all)   

     


EEG_Deep/Data2A/parsed_P01T.mat
Filtering of Training Data Finished
EEG_Deep/Data2A/parsed_P01E.mat
Filtering of Testing Data Finished
(1, 2)
Finding labels in training data
Finding labels in testing data
[1 1 1 2 1 2 2 1 2 2 2 1 1 1 1 2 2 1 2 1]
(60, 12, 1536)
(60,)
(20, 12, 1536)
(20,)
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
X_train shape: (60, 1, 12, 1536)
60 train samples
20 val samples
(40, 12, 1536)
(40, 1)
X_train shape: (40, 1, 12, 1536)
40 train samples
Train on 2 steps
Epoch 1/100
2/2 [==============================] - 0s 111ms/step - loss: 1.4897 - categorical_crossentropy: 1.4897 - categorical_accuracy: 0.2667
Epoch 2/100
2/2 [==============================] - 0s 17ms/step - loss: 1.2531 - categorical_crossentropy: 1.

ValueError: ignored

In [0]:
import pandas as pd
df_accl_all = pd.DataFrame({'Class1vs2': acc_all[:, 0]})
df_accl_all.to_csv (r'EEG_Deep/df_accl_allPatient_8_24_2560:4096.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
